# **1) Clean Data**

---

##### *À quoi ressemble la data extraite par le webscraper ?*

In [1]:
import pandas as pd
import re

raw_data_louer = pd.read_csv('2_raw_data/Immobilier_ch_louer.csv')
raw_data_louer.head()

,web-scraper-order,web-scraper-start-url,price_text,metres_carres,ville_adresse,type_pieces
0,1699260062-1,https://www.immobilier.ch/fr/carte/louer/appar...,CHF 990.-/mois (+ 260.- Charges),72 m2,"Bellach, Grederstrasse 18",Appartement 3.5 pièces
1,1699260062-2,https://www.immobilier.ch/fr/carte/louer/appar...,CHF 2'600.-/mois,30 m2,"Genève, Rue de l'Athenee",Appartement meublé
2,1699260062-3,https://www.immobilier.ch/fr/carte/louer/appar...,CHF 3'940.-/mois,30 m2,"Paradiso, Via G.b. Dominione",Appartement meublé 1 pièce
3,1699260062-4,https://www.immobilier.ch/fr/carte/louer/appar...,MON 3'880.-/mois (+ 3'880.- Charges),NaN,"Zürich, Zweierstrasse",Appartement meublé 2 pièces
4,1699260062-5,https://www.immobilier.ch/fr/carte/louer/appar...,CHF 3'185.-/mois,30 m2,"Zürich, Baslerstrasse",Appartement meublé 1 pièce


---

##### **Definition de la fonction Clean_Data**

In [2]:
def clean_data(file_path, filepath2):
    #Charge le fichier CSV
    data = pd.read_csv(file_path)
    loca_comm = pd.read_csv(filepath2)

    #Supprime les deux premières colonnes
    data = data.iloc[:,2:]

    #Change l'ordre des colonnes
    new_order = ['ville_adresse', 'price_text', 'metres_carres', 'type_pieces']
    data = data.reindex(columns = new_order)

    #Enlève les lignes 'metres_carres' vides (mètres carrés non spécifiés dans l'annonce)
    data = data.dropna(subset=['metres_carres'])

    #Supprime les lignes où 'price_text' ne contient pas de chiffres (prix non spécifié dans l'annonce,'Loyer sur demande')
    data = data[data['price_text'].str.contains(r'\d', na=False)]

    #Supprime le m2 dans la colonne 'metres_carres'
    data['metres_carres'] = data['metres_carres'].apply(lambda x: re.sub(r'(\d+)\s*m2', r'\1', x))

    #Converti la colonne 'metres_carres' de type str en int
    data['metres_carres'] = data['metres_carres'].astype(int)

    #Supprime le code postal s'il y en a un, Sépare 'ville_adresse' sur la virgule et récupère le premier élément (la ville)
    data['ville_adresse'] = data['ville_adresse'].apply(lambda x: re.sub(r'^\d+\s', '', x.split(',')[0]))

    #Extrait le premier nombre de la colonne 'price_text' (prix sans les charges) et le convertit en entier.
    data['price_text'] = data['price_text'].str.extract(r'(\d[\d\']*)')
    data['price_text'] = data['price_text'].str.replace(r"'", "", regex=True).astype(int)

    #Crée une nouvelle colonne 'nbr_pieces' si 'type_pieces' possède un chiffre et le convertit en float.
    data['nbr_pieces'] = data['type_pieces'].str.extract(r'(\d+\.?\d*)', expand=False).astype(float)

    #Supprime tout ce qui suit un chiffre dans la colonne 'type_pieces' (ne garde que le type)
    data['type_pieces'] = data['type_pieces'].str.replace(r'\d.*', '', regex=True)

    #Renomme les colonnes avec ce qu'elles contiennent après le nettoyage
    data = data.rename(columns={'ville_adresse': 'ville', 'price_text': 'prix', 'type_pieces': 'type'})


    #REGROUPE LES VILLAGES AVEC LES COMMUNES (pour éviter de créer pleins de petites formes pour la map)
    #Gère les erreurs
    data['ville'] = data['ville'].replace('Biel-Bienne', 'Biel/Bienne')
    data['ville'] = data['ville'].replace('Morat', 'Murten')
    data['ville'] = data['ville'].replace('Ormône (Savièse)', 'Savièse')
    data['ville'] = data['ville'].replace('Rosé', 'Avry-sur-Matran')
    data['ville'] = data['ville'].replace('Ayent', 'Anzère')
    data['ville'] = data['ville'].replace('Granois (Savièse)', 'Savièse')
    data['ville'] = data['ville'].replace('Drône', 'Savièse')
    data['ville'] = data['ville'].replace('Thyon-Les Collons', 'Vex')
    data['ville'] = data['ville'].replace('Breil-Brigels', 'Breil/Brigels')
    data['ville'] = data['ville'].replace('Bardonnex', 'Perly')
    data['ville'] = data['ville'].replace('Brigue', 'Brig')
    data['ville'] = data['ville'].replace('Saint-Germain (Savièse)', 'Savièse')
    data['ville'] = data['ville'].replace('Montagnier (Le Châble)', 'Chemin')
    data['ville'] = data['ville'].replace('Marin-Centre', 'Marin-Epagnier')
    data['ville'] = data['ville'].replace('Prarreyer (Versegères)', 'Versegères')
    data['ville'] = data['ville'].replace('Chandolin-près-Savièse', 'Savièse')
    data['ville'] = data['ville'].replace('Fontenelle (Le Châble)', 'Chemin')
    data['ville'] = data['ville'].replace('La Montoz (Versegères)', 'Versegères')
    data['ville'] = data['ville'].replace('Waltensburg-Vuorz', 'Breil/Brigels')
    data['ville'] = data['ville'].replace('Mâche', 'Hérémence')
    data['ville'] = data['ville'].replace('Villette (Le Châble)', 'Chemin')
    data['ville'] = data['ville'].replace('Feldis-Veulden', 'Feldis/Veulden')
    data['ville'] = data['ville'].replace('Coméraz (Grimisuat)', 'Grimisuat')
    data['ville'] = data['ville'].replace('Domat-Ems', 'Domat/Ems')
    data['ville'] = data['ville'].replace('Disentis-Mustér', 'Disentis/Mustér')
    #Crée un dictionnaire de correspondance entre 'ville' et 'commune'
    correspondance_dict = dict(zip(loca_comm['localite'], loca_comm['commune']))
    #Filtre et affiche les lignes où 'ville' ne correspond à aucune 'localite'
    villes_non_correspondantes = data.loc[~data['ville'].isin(loca_comm['localite']), 'ville'].unique()
    print(villes_non_correspondantes)
    #Applique la correspondance uniquement aux lignes où 'ville' correspond à une 'localite'
    data['ville'] = data.apply(lambda row: correspondance_dict.get(row['ville'], row['ville']), axis=1)

    #CRÉE COLONNE CANTON
    data['canton'] = ''
    #Crée un dictionnaire de correspondance entre 'commune' et 'canton' de loca_comm
    correspondance_dict2 = dict(zip(loca_comm['commune'], loca_comm['canton']))
    #Applique la correspondance à la colonne 'ville' de votre DataFrame 'data'
    data['canton'] = data['ville'].map(correspondance_dict2)

    #Supprime le VD, GE,... dans le nom de la ville. (Supprime le deuxième mot si la ville est en 2 mots et que le deuxième mot est en maj.)
    data['ville'] = data['ville'].apply(lambda x: x.split(' ')[0] if len(x.split(' ')) == 2 and x.split(' ')[1].isupper() else x)

    #Remplace St- en Saint et Ste- en Sainte
    data['ville'] = data['ville'].apply(lambda x: x.replace('St-', 'Saint-') if x.startswith('St-') else x)
    data['ville'] = data['ville'].apply(lambda x: x.replace('Ste-', 'Sainte-') if x.startswith('Ste-') else x)

    # Réorganise les colonnes pour mettre 'canton' en deuxième position
    data = data[['ville', 'canton', 'prix', 'metres_carres', 'type', 'nbr_pieces']]
    #Réindexe les lignes sans les lignes supprimées (et supprime l'ancienne colonne d'index incorrecte)
    data.reset_index(drop=True, inplace=True)

    
    return data



---

### ***Clean_data Immobilier.ch Louer***

##### <span style='color:red'>**Modifier data ici ↓**</span>

In [3]:
cleaned_data_louer0 = clean_data('2_raw_data/Immobilier_ch_louer.csv','3_localite_commune/Clean_localite_commune.csv')

#Enlève les annonces des gens qui ont mis une annonce de vente dans les pages de location
def clean_listing_errors_louer(dataframe, threshold=100000):
    filtered_data = dataframe.query('prix <= @threshold')
    return filtered_data


cleaned_data_louer = clean_listing_errors_louer(cleaned_data_louer0)
cleaned_data_louer.head(20)

[]


,ville,canton,prix,metres_carres,type,nbr_pieces
0,Selzach,SO,990,72,Appartement,3.5
1,Vernier,GE,2600,30,Appartement meublé,NaN
2,Paradiso,TI,3940,30,Appartement meublé,1.0
3,Zürich,ZH,3185,30,Appartement meublé,1.0
4,Vernier,GE,2650,48,Appartement,3.0
5,Romanel-sur-Lausanne,VD,1960,82,Appartement,3.5
6,Neuchâtel,NE,2490,150,Villa,6.0
7,Lausanne,VD,2054,69,Appartement,3.5
8,Saint-Sulpice,VD,1800,44,Appartement,2.0
9,Lausanne,VD,1903,74,Appartement,3.5


---

In [4]:
#Permet de voir si des gens ont fait des erreurs et ont posté leur annonce de vente dans la section location
'''
def top_10_prices(dataframe):
    # Trie le DataFrame par la colonne 'prix' de manière décroissante et récupère les 10 premières lignes
    top_prices = dataframe.nlargest(10, 'prix')
    return top_prices


top_10_prices_data = top_10_prices(cleaned_data_louer0)
print(top_10_prices_data)
'''

"\ndef top_10_prices(dataframe):\n    # Trie le DataFrame par la colonne 'prix' de manière décroissante et récupère les 10 premières lignes\n    top_prices = dataframe.nlargest(10, 'prix')\n    return top_prices\n\n\ntop_10_prices_data = top_10_prices(cleaned_data_louer0)\nprint(top_10_prices_data)\n"

In [5]:
#Création du nouveau fichier CSV nettoyé
cleaned_data_louer.to_csv('1_useful_data/1_cleaned_data/Cleaned_Immobilier_ch_louer.csv', index=False)

---

### ***Clean_data Immobilier.ch Acheter***

In [6]:
raw_data_acheter = pd.read_csv('2_raw_data/Immobilier_ch_acheter.csv')
raw_data_acheter.head()

,web-scraper-order,web-scraper-start-url,price_text,metres_carres,ville_adresse,type_pieces
0,1699274286-1,https://www.immobilier.ch/fr/carte/acheter/app...,CHF 2'650'000.-,180 m2,"1884 Villars-sur-Ollon, Contactez l'agence pou...",Chalet 5 pièces
1,1699274286-2,https://www.immobilier.ch/fr/carte/acheter/app...,CHF 555'000.-,87 m2,"1912 Leytron, Contactez l'agence pour l'adresse",Appartement 3.5 pièces
2,1699274286-3,https://www.immobilier.ch/fr/carte/acheter/app...,CHF 3'500'000.-,146 m2,"Verbier, Chemin de Médières",Appartement 5.5 pièces
3,1699274286-4,https://www.immobilier.ch/fr/carte/acheter/app...,Prix sur demande,NaN,"1964 Conthey, Contactez l'agence pour l'adresse",Appartement
4,1699274286-5,https://www.immobilier.ch/fr/carte/acheter/app...,CHF 755'000.-,146 m2,"1961 Vernamiège, Contactez l'agence pour l'adr...",Chalet 5.5 pièces


##### <span style='color:red'>**Modifier data ici ↓**</span>

In [7]:
cleaned_data_acheter0 = clean_data('2_raw_data/Immobilier_ch_acheter.csv','3_localite_commune/Clean_localite_commune.csv')

#Enlève les annonces des gens qui ont mis une annonce de location dans les pages de vente
def clean_listing_errors_acheter(dataframe, threshold=30000):
    filtered_data = dataframe.query('prix >= @threshold')
    return filtered_data


cleaned_data_acheter = clean_listing_errors_acheter(cleaned_data_acheter0)
cleaned_data_acheter.head(20)

[]


,ville,canton,prix,metres_carres,type,nbr_pieces
0,Ollon,VD,2650000,180,Chalet,5.0
1,Leytron,VS,555000,87,Appartement,3.5
2,Val de Bagnes,VS,3500000,146,Appartement,5.5
3,Mont-Noble,VS,755000,146,Chalet,5.5
4,Liestal,BL,1840000,150,Appartement,4.5
5,Chamoson,VS,1190000,227,Villa,6.5
6,La Tène,NE,715000,93,Appartement,4.5
7,Ollon,VD,2295000,171,Duplex,4.5
8,La Sarraz,VD,2900000,530,Immeuble résidentiel,18.0
9,Crans-Montana,VS,2500000,183,Appartement,5.0


---

In [8]:
#Permet de voir si des gens ont fait des erreurs et ont posté leur annonce de location dans la section vente
'''
def bottom_10_prices(dataframe):
    # Trie le DataFrame par la colonne 'prix' de manière décroissante et récupère les 10 premières lignes
    top_prices = dataframe.nsmallest(10, 'prix')
    return top_prices


bottom_10_prices_data = bottom_10_prices(cleaned_data_acheter0)
print(bottom_10_prices_data)
'''

"\ndef bottom_10_prices(dataframe):\n    # Trie le DataFrame par la colonne 'prix' de manière décroissante et récupère les 10 premières lignes\n    top_prices = dataframe.nsmallest(10, 'prix')\n    return top_prices\n\n\nbottom_10_prices_data = bottom_10_prices(cleaned_data_acheter0)\nprint(bottom_10_prices_data)\n"

In [9]:
#Création du nouveau fichier CSV nettoyé
cleaned_data_acheter.to_csv('1_useful_data/1_cleaned_data/Cleaned_Immobilier_ch_acheter.csv', index=False)

ㅤ

---